variables que vamos a escoger para el modelo
variables que se quieran discretizar
en la carpeta modelo se puede comparar modelos, ejemplo modelo de regresion, modelo de arbol o solo uno pero explicar por que se selecciono